# Trends of Power Outages

**Name(s)**: Anirudh Indraganti (A16032124)

**Website Link**: https://aindragaofficial.github.io/PowerOutageAnalysis/

Question: Are there similar amount of power outages across different regions?

## Code

In [94]:
import pandas as pd
import numpy as np
import os
import folium
import geopandas as gpd
import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Data Cleaning

In [95]:
power_outages_df = pd.read_excel('outage.xlsx', header=5)
power_outages_df.head()

,variables,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,Units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,numeric,NaN,...,%,%,persons per square mile,persons per square mile,persons per square mile,%,%,%,%,%
1,NaN,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,NaN,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,NaN,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,NaN,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743


In [96]:
power_outages_df.drop(['variables', 'OBS'], axis=1, inplace=True)
power_outages_df.drop(0, inplace=True)
print(power_outages_df.shape)
power_outages_df.head()

(1534, 55)


,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,17:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,04:30:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
5,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,02:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743


In [97]:
clean_power_outages_df = power_outages_df.assign(
    **{
        'OUTAGE.START.TIME': pd.to_datetime(power_outages_df['OUTAGE.START.DATE']) + pd.to_timedelta(power_outages_df['OUTAGE.START.TIME'].astype(str, errors='ignore')),
        'OUTAGE.RESTORATION.TIME': pd.to_datetime(power_outages_df['OUTAGE.RESTORATION.DATE']) + pd.to_timedelta(power_outages_df['OUTAGE.RESTORATION.TIME'].astype(str, errors='ignore')),
        'OUTAGE.DURATION': power_outages_df['OUTAGE.DURATION'] / 60
    }
).drop(['OUTAGE.START.DATE', 'OUTAGE.RESTORATION.DATE'], axis=1, inplace=False)

clean_power_outages_df.reset_index(inplace=True)
not_drop_cols = [
    'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'OUTAGE.START.TIME',\
    'OUTAGE.RESTORATION.TIME', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.CUSTOMERS',\
    'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'CAUSE.CATEGORY'
 ]
clean_power_outages_df.drop(clean_power_outages_df.columns.difference(not_drop_cols), axis=1, inplace=True)
clean_power_outages_df.head()

,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,OUTAGE.START.TIME,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS
0,Minnesota,MN,MRO,East North Central,-0.3,2011-07-01 17:00:00,2011-07-03 20:00:00,severe weather,51.0,NaN,70000.0,2308736.0,276286.0,10673.0,2595696.0
1,Minnesota,MN,MRO,East North Central,-0.1,2014-05-11 18:38:00,2014-05-11 18:39:00,intentional attack,0.016667,NaN,NaN,2345860.0,284978.0,9898.0,2640737.0
2,Minnesota,MN,MRO,East North Central,-1.5,2010-10-26 20:00:00,2010-10-28 22:00:00,severe weather,50.0,NaN,70000.0,2300291.0,276463.0,10150.0,2586905.0
3,Minnesota,MN,MRO,East North Central,-0.1,2012-06-19 04:30:00,2012-06-20 23:00:00,severe weather,42.5,NaN,68200.0,2317336.0,278466.0,11010.0,2606813.0
4,Minnesota,MN,MRO,East North Central,1.2,2015-07-18 02:00:00,2015-07-19 07:00:00,severe weather,29.0,250,250000.0,2374674.0,289044.0,9812.0,2673531.0


## Univariate Analysis

In [98]:
fig = px.histogram(clean_power_outages_df, x='OUTAGE.DURATION')
fig

In [99]:
px.histogram(clean_power_outages_df, x='DEMAND.LOSS.MW')

## Bivariate Analysis

In [100]:
geojson_data = gpd.read_file('usa_map.json')
geojson_data.head()

,id,name,geometry
0,AL,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,AK,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,AZ,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
3,AR,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
4,CA,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."


In [101]:
avg_outage_durations = clean_power_outages_df.groupby('U.S._STATE')['OUTAGE.DURATION'].mean().to_frame().reset_index(inplace=False)
avg_outage_durations.head()

,U.S._STATE,OUTAGE.DURATION
0,Alabama,19.213333
1,Alaska,NaN
2,Arizona,75.882000
3,Arkansas,25.239333
4,California,27.772306


In [102]:
usa_map = folium.Map(location=[40, -95], zoom_start=4)
folium.Choropleth(
    geo_data=geojson_data,
    data=avg_outage_durations,
    columns=['U.S._STATE', 'OUTAGE.DURATION'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Outage Duration',
).add_to(usa_map)

folium.LayerControl().add_to(usa_map)
usa_map

In [103]:
months_df = clean_power_outages_df.assign(
    **{
        'MONTH': clean_power_outages_df['OUTAGE.START.TIME'].dt.month
    }
)

months_df = months_df[months_df['OUTAGE.DURATION'].notna()]
months_df.head()

,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,OUTAGE.START.TIME,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,MONTH
0,Minnesota,MN,MRO,East North Central,-0.3,2011-07-01 17:00:00,2011-07-03 20:00:00,severe weather,51.0,NaN,70000.0,2308736.0,276286.0,10673.0,2595696.0,7.0
1,Minnesota,MN,MRO,East North Central,-0.1,2014-05-11 18:38:00,2014-05-11 18:39:00,intentional attack,0.016667,NaN,NaN,2345860.0,284978.0,9898.0,2640737.0,5.0
2,Minnesota,MN,MRO,East North Central,-1.5,2010-10-26 20:00:00,2010-10-28 22:00:00,severe weather,50.0,NaN,70000.0,2300291.0,276463.0,10150.0,2586905.0,10.0
3,Minnesota,MN,MRO,East North Central,-0.1,2012-06-19 04:30:00,2012-06-20 23:00:00,severe weather,42.5,NaN,68200.0,2317336.0,278466.0,11010.0,2606813.0,6.0
4,Minnesota,MN,MRO,East North Central,1.2,2015-07-18 02:00:00,2015-07-19 07:00:00,severe weather,29.0,250,250000.0,2374674.0,289044.0,9812.0,2673531.0,7.0


In [104]:
lower_bound = np.quantile(months_df['OUTAGE.DURATION'], 0.75)
major_outages = months_df.loc[months_df['OUTAGE.DURATION'] >= lower_bound].groupby('U.S._STATE')['OUTAGE.DURATION'].count().to_frame()
major_outages.reset_index(inplace=True)
major_outages.columns = ['U.S State', 'Major Outages']

new_usa_map = folium.Map(location=[40, -95], zoom_start=4)
folium.Choropleth(
    geo_data=geojson_data,
    data=major_outages,
    columns=['U.S State', 'Major Outages'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Major Outages',
).add_to(new_usa_map)

folium.LayerControl().add_to(new_usa_map)
new_usa_map

## Interesting Aggregates

In [105]:
pd.pivot_table(
    clean_power_outages_df, 'OUTAGE.DURATION',
    'CLIMATE.REGION', 'CAUSE.CATEGORY', 'mean'
)

CAUSE.CATEGORY,equipment failure,fuel supply emergency,intentional attack,islanding,public appeal,severe weather,system operability disruption
CLIMATE.REGION,,,,,,,
Central,5.366667,167.254167,5.767647,2.088889,23.500000,54.166792,44.920000
East North Central,440.588889,566.187500,39.600833,0.016667,12.216667,73.913622,43.500000
Northeast,3.596667,243.826190,3.266412,14.683333,44.250000,73.831714,12.891667
Northwest,11.700000,0.016667,6.230196,1.222222,14.966667,80.633333,2.350000
South,4.929630,291.375000,5.426786,8.225000,19.399603,73.189151,14.434568
Southeast,9.241667,NaN,8.411111,NaN,47.756667,44.376006,2.821875
Southwest,1.896667,1.266667,4.427869,0.033333,37.916667,192.881667,5.487037
West,8.746825,102.576667,14.294624,3.580952,33.801852,48.806219,6.061111
West North Central,1.016667,NaN,0.391667,1.136667,7.325000,40.708333,NaN


### Assessment of Missingness

## Missingness Dependency

In [106]:
state_dist = clean_power_outages_df.assign(**{
    'CLIMATE.REGION.MISSING': clean_power_outages_df['CLIMATE.REGION'].isna()
}).pivot_table(
    index='U.S._STATE', columns='CLIMATE.REGION.MISSING', aggfunc='size'
).replace(np.NaN, 0, inplace=False)

state_dist.columns = ['climate_region_missing = false', 'climate_region_missing = true']
state_dist = state_dist / state_dist.sum()
state_dist.head()

,climate_region_missing = false,climate_region_missing = true
U.S._STATE,,
Alabama,0.003927,0.000000
Alaska,0.000000,0.166667
Arizona,0.018325,0.000000
Arkansas,0.016361,0.000000
California,0.137435,0.000000


In [107]:
fig = state_dist.plot(kind='barh', title='State by Climate Region Missingness', barmode='group')
fig

In [108]:
fig.write_html('state_climate_missing.html')

In [109]:
repitions = 500
shuffled = clean_power_outages_df.copy(deep=True)
shuffled['CLIMATE.REGION.MISSING'] = shuffled['CLIMATE.REGION'].isna()
shuffled

,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,OUTAGE.START.TIME,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,CLIMATE.REGION.MISSING
0,Minnesota,MN,MRO,East North Central,-0.3,2011-07-01 17:00:00,2011-07-03 20:00:00,severe weather,51.0,NaN,70000.0,2308736.0,276286.0,10673.0,2595696.0,False
1,Minnesota,MN,MRO,East North Central,-0.1,2014-05-11 18:38:00,2014-05-11 18:39:00,intentional attack,0.016667,NaN,NaN,2345860.0,284978.0,9898.0,2640737.0,False
2,Minnesota,MN,MRO,East North Central,-1.5,2010-10-26 20:00:00,2010-10-28 22:00:00,severe weather,50.0,NaN,70000.0,2300291.0,276463.0,10150.0,2586905.0,False
3,Minnesota,MN,MRO,East North Central,-0.1,2012-06-19 04:30:00,2012-06-20 23:00:00,severe weather,42.5,NaN,68200.0,2317336.0,278466.0,11010.0,2606813.0,False
4,Minnesota,MN,MRO,East North Central,1.2,2015-07-18 02:00:00,2015-07-19 07:00:00,severe weather,29.0,250,250000.0,2374674.0,289044.0,9812.0,2673531.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,North Dakota,ND,MRO,West North Central,-0.9,2011-12-06 08:00:00,2011-12-06 20:00:00,public appeal,12.0,155,34500.0,330738.0,60017.0,3639.0,394394.0,False
1530,North Dakota,ND,MRO,West North Central,NaN,NaT,NaT,fuel supply emergency,NaN,1650,NaN,309997.0,53709.0,2331.0,366037.0,False
1531,South Dakota,SD,RFC,West North Central,0.5,2009-08-29 22:54:00,2009-08-29 23:53:00,islanding,0.983333,84,NaN,367206.0,65971.0,3052.0,436229.0,False
1532,South Dakota,SD,MRO,West North Central,0.5,2009-08-29 11:00:00,2009-08-29 14:01:00,islanding,3.016667,373,NaN,367206.0,65971.0,3052.0,436229.0,False


In [110]:
tvds = []
for _ in range(repitions):
    shuffled['U.S._STATE'] = np.random.permutation(shuffled['U.S._STATE'])
    pivoted = shuffled.pivot_table(
        index='U.S._STATE', columns='CLIMATE.REGION.MISSING', aggfunc='size'
    ).apply(lambda x: x / x.sum()).replace(np.NaN, 0, inplace=False)

    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)

In [111]:
observed_tvd = state_dist.diff(axis=1).iloc[:, -1].abs().sum() / 2
observed_tvd

1.0

In [112]:
fig = px.histogram(pd.DataFrame(tvds), x=0, nbins=50, histnorm='probability', 
                   title='Emp Dist of TVD')
fig.add_vline(x=observed_tvd, line_color='red')
fig

In [113]:
p_value = np.mean(np.array(tvds) >= observed_tvd)
p_value

0.0

Reject null and say that data too extreme to say missing climate region data with respect to US State is MCAR

In [114]:
cause_dist = clean_power_outages_df.assign(**{
    'outage_missing': clean_power_outages_df['OUTAGE.START.TIME'].isna()
}).pivot_table(
    index='CAUSE.CATEGORY', columns='outage_missing', aggfunc='size'
).replace(np.NaN, 0, inplace=False)

cause_dist.columns = ['outage_missing = false', 'outage_missing = true']
cause_dist = cause_dist / cause_dist.sum()
cause_dist.head()

,outage_missing = false,outage_missing = true
CAUSE.CATEGORY,,
equipment failure,0.037377,0.333333
fuel supply emergency,0.032787,0.111111
intentional attack,0.274098,0.000000
islanding,0.030164,0.000000
public appeal,0.045246,0.000000


In [115]:
fig = cause_dist.plot(kind='barh', title='Cause Category by Demand Loss Missingness', barmode='group')
fig

In [116]:
repitions = 500
shuffled = clean_power_outages_df.copy(deep=True)
shuffled['outage_missing'] = shuffled['OUTAGE.START.TIME'].isna()
shuffled.head()

,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,OUTAGE.START.TIME,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,outage_missing
0,Minnesota,MN,MRO,East North Central,-0.3,2011-07-01 17:00:00,2011-07-03 20:00:00,severe weather,51.0,NaN,70000.0,2308736.0,276286.0,10673.0,2595696.0,False
1,Minnesota,MN,MRO,East North Central,-0.1,2014-05-11 18:38:00,2014-05-11 18:39:00,intentional attack,0.016667,NaN,NaN,2345860.0,284978.0,9898.0,2640737.0,False
2,Minnesota,MN,MRO,East North Central,-1.5,2010-10-26 20:00:00,2010-10-28 22:00:00,severe weather,50.0,NaN,70000.0,2300291.0,276463.0,10150.0,2586905.0,False
3,Minnesota,MN,MRO,East North Central,-0.1,2012-06-19 04:30:00,2012-06-20 23:00:00,severe weather,42.5,NaN,68200.0,2317336.0,278466.0,11010.0,2606813.0,False
4,Minnesota,MN,MRO,East North Central,1.2,2015-07-18 02:00:00,2015-07-19 07:00:00,severe weather,29.0,250,250000.0,2374674.0,289044.0,9812.0,2673531.0,False


In [117]:
tvds = []
for _ in range(repitions):
    shuffled['CAUSE.CATEGORY'] = np.random.permutation(shuffled['CAUSE.CATEGORY'])
    pivoted = shuffled.pivot_table(
        index='CAUSE.CATEGORY', columns='outage_missing', aggfunc='size'
    ).apply(lambda x: x / x.sum()).replace(np.NaN, 0, inplace=False)

    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)

In [118]:
observed_tvd = cause_dist.diff(axis=1).iloc[:, -1].abs().sum() / 2
observed_tvd

0.40276867030965396

In [119]:
fig = px.histogram(pd.DataFrame(tvds), x=0, nbins=50, histnorm='probability', 
                   title='Emp Dist of TVD')
fig.add_vline(x=observed_tvd, line_color='red')
fig

In [120]:
p_value = np.mean(np.array(tvds) >= observed_tvd)
p_value

0.058

We fail to reject null. Data is not extreme enough to say data is not MCAR. Therefore, we support the null hypothesis.

### Hypothesis Testing

In [121]:
clean_power_outages_df

,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,OUTAGE.START.TIME,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS
0,Minnesota,MN,MRO,East North Central,-0.3,2011-07-01 17:00:00,2011-07-03 20:00:00,severe weather,51.0,NaN,70000.0,2308736.0,276286.0,10673.0,2595696.0
1,Minnesota,MN,MRO,East North Central,-0.1,2014-05-11 18:38:00,2014-05-11 18:39:00,intentional attack,0.016667,NaN,NaN,2345860.0,284978.0,9898.0,2640737.0
2,Minnesota,MN,MRO,East North Central,-1.5,2010-10-26 20:00:00,2010-10-28 22:00:00,severe weather,50.0,NaN,70000.0,2300291.0,276463.0,10150.0,2586905.0
3,Minnesota,MN,MRO,East North Central,-0.1,2012-06-19 04:30:00,2012-06-20 23:00:00,severe weather,42.5,NaN,68200.0,2317336.0,278466.0,11010.0,2606813.0
4,Minnesota,MN,MRO,East North Central,1.2,2015-07-18 02:00:00,2015-07-19 07:00:00,severe weather,29.0,250,250000.0,2374674.0,289044.0,9812.0,2673531.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,North Dakota,ND,MRO,West North Central,-0.9,2011-12-06 08:00:00,2011-12-06 20:00:00,public appeal,12.0,155,34500.0,330738.0,60017.0,3639.0,394394.0
1530,North Dakota,ND,MRO,West North Central,NaN,NaT,NaT,fuel supply emergency,NaN,1650,NaN,309997.0,53709.0,2331.0,366037.0
1531,South Dakota,SD,RFC,West North Central,0.5,2009-08-29 22:54:00,2009-08-29 23:53:00,islanding,0.983333,84,NaN,367206.0,65971.0,3052.0,436229.0
1532,South Dakota,SD,MRO,West North Central,0.5,2009-08-29 11:00:00,2009-08-29 14:01:00,islanding,3.016667,373,NaN,367206.0,65971.0,3052.0,436229.0


In [122]:
cleaned_outage_duration = clean_power_outages_df[clean_power_outages_df['OUTAGE.DURATION'].notna()]
cleaned_major_outages = cleaned_outage_duration.loc[cleaned_outage_duration['OUTAGE.DURATION'] >= np.quantile(cleaned_outage_duration['OUTAGE.DURATION'], 0.75)].\
groupby('NERC.REGION')['OUTAGE.DURATION'].count()

print(cleaned_major_outages.sum())

cleaned_major_outages = cleaned_major_outages / cleaned_major_outages.sum()
cleaned_major_outages

377


NERC.REGION
ECAR    0.076923
FRCC    0.045093
MRO     0.034483
NPCC    0.122016
RFC     0.416446
SERC    0.074271
SPP     0.039788
TRE     0.066313
WECC    0.124668
Name: OUTAGE.DURATION, dtype: float64

Null Hypothesis: There is no difference in proportions of major power outages between NERC regions.

Alternative Hypothesis: There is a difference in proportions of major power outages between NERC regions.

We define a major power outage as being the 75th percentile or above in outage duration.

In [123]:
hypothesis_df = cleaned_major_outages.to_frame()
hypothesis_df['Expected Proportions'] = [(1 / hypothesis_df.shape[0]) for _ in range(hypothesis_df.shape[0])]
hypothesis_df.columns = ['Observed Proportions', 'Expected Proportions']
hypothesis_df

,Observed Proportions,Expected Proportions
NERC.REGION,,
ECAR,0.076923,0.111111
FRCC,0.045093,0.111111
MRO,0.034483,0.111111
NPCC,0.122016,0.111111
RFC,0.416446,0.111111
SERC,0.074271,0.111111
SPP,0.039788,0.111111
TRE,0.066313,0.111111
WECC,0.124668,0.111111


In [124]:
hypothesis_df.plot(kind='barh', title='Distribution of Proportions of Major Outages Across NERC Regions', barmode='group')

In [125]:
observed_tvd = np.sum(np.abs(hypothesis_df['Observed Proportions'] - hypothesis_df['Expected Proportions'])) / 2
observed_tvd

0.32979664014146765

In [126]:
N_Outages = 377
draws = np.random.multinomial(N_Outages, hypothesis_df['Expected Proportions'], size=100_000) / N_Outages
draws

array([[0.09549072, 0.10079576, 0.09549072, ..., 0.14058355, 0.12466844,
        0.12201592],
       [0.12997347, 0.0928382 , 0.11405836, ..., 0.10079576, 0.1061008 ,
        0.08753316],
       [0.0928382 , 0.13793103, 0.15384615, ..., 0.10079576, 0.10079576,
        0.09018568],
       ...,
       [0.10875332, 0.11405836, 0.09814324, ..., 0.12997347, 0.13793103,
        0.0795756 ],
       [0.10344828, 0.1193634 , 0.10875332, ..., 0.12201592, 0.11671088,
        0.10079576],
       [0.12732095, 0.10079576, 0.09814324, ..., 0.09549072, 0.12201592,
        0.1061008 ]])

In [127]:
all_tvds = np.sum(np.abs(draws - hypothesis_df['Expected Proportions'].to_numpy()), axis=1) / 2
all_tvds

array([0.0542293 , 0.05717654, 0.07810197, ..., 0.06248158, 0.04656646,
       0.05688182])

In [128]:
fig = px.histogram(pd.DataFrame(all_tvds), x=0, nbins=50, histnorm='probability', 
                   title='Empirical Distribution of the TVD')
fig.add_vline(x=observed_tvd, line_color='red')
fig

In [129]:
p_value = (np.array(all_tvds) >= observed_tvd).mean()
p_value

0.0

Conclusion: We reject the null hypothesis and support the claim that the proportion of major outages is different across the different NERC regions in some way.

This was relatively expected just from viewing the data in the grouped bar chart but our p value confirms our results.